# Entraînement du Neural Network pour Smart Chess sur Google Colab

Ce notebook permet d'entraîner le réseau de neurones pour l'évaluation d'échecs en utilisant les ressources GPU de Google Colab.

**Chemin du projet sur Drive:** `MyDrive/smart_chess_drive/smart-chess`

## Instructions
1. Aller dans **Runtime > Change runtime type > GPU** (T4 ou mieux)
2. Exécuter les cellules dans l'ordre
3. Les modèles seront sauvegardés automatiquement sur votre Drive

## 1. Vérification GPU

In [1]:
# Vérifier la disponibilité du GPU
!nvidia-smi

Tue Oct 28 09:14:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   32C    P8             11W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 2. Montage Google Drive

In [2]:
# Monter Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 3. Configuration du chemin du projet

In [3]:
# Définir le chemin vers le projet sur votre Drive
import os
import sys

PROJECT_PATH = '/content/drive/MyDrive/smart_chess_drive/smart-chess'
os.chdir(PROJECT_PATH)
sys.path.insert(0, PROJECT_PATH)

print(f"Répertoire de travail: {os.getcwd()}")
print(f"\nContenu du répertoire:")
for item in sorted(os.listdir('.')):
    print(f"  - {item}")

Répertoire de travail: /content/drive/MyDrive/smart_chess_drive/smart-chess

Contenu du répertoire:
  - .git
  - .gitignore
  - README.md
  - ai
  - docs
  - prototypes


## 4. Installation des dépendances

In [4]:
# Installer les packages nécessaires
!pip install -q torch torchvision torchaudio
!pip install -q numpy matplotlib tqdm

print("✓ Installation terminée")

✓ Installation terminée


## 5. Vérification de l'environnement PyTorch

In [5]:
import torch
import numpy as np

print("=" * 60)
print("CONFIGURATION SYSTÈME")
print("=" * 60)
print(f"PyTorch version: {torch.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"\nCUDA disponible: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Nom du GPU: {torch.cuda.get_device_name(0)}")
    props = torch.cuda.get_device_properties(0)
    print(f"Mémoire GPU totale: {props.total_memory / 1e9:.2f} GB")
    print(f"Compute Capability: {props.major}.{props.minor}")
else:
    print("⚠️ ATTENTION: GPU non disponible, l'entraînement sera très lent!")
    print("   Allez dans Runtime > Change runtime type > GPU")

print("=" * 60)

CONFIGURATION SYSTÈME
PyTorch version: 2.8.0+cu126
NumPy version: 2.0.2

CUDA disponible: True
CUDA version: 12.6
Nom du GPU: NVIDIA L4
Mémoire GPU totale: 23.80 GB
Compute Capability: 8.9


## 6. Import des modules du projet

In [6]:
# Importer les modules nécessaires depuis le projet (robuste à l'emplacement du repo sur Drive)
import os
import sys
import importlib

# Assurez-vous que PROJECT_PATH est défini et ajoutez également le dossier `ai` au PYTHONPATH
PROJECT_PATH = '/content/drive/MyDrive/smart_chess_drive/smart-chess'
AI_SUBDIR = os.path.join(PROJECT_PATH, 'ai')

# Vérifier les chemins alternatifs (si l'utilisateur a copié le repo dans /content)
ALT_PATH = '/content/smart-chess'

# Choisir un chemin existant
if not os.path.isdir(PROJECT_PATH) and os.path.isdir(ALT_PATH):
    PROJECT_PATH = ALT_PATH

if not os.path.isdir(PROJECT_PATH):
    raise FileNotFoundError(f"Répertoire projet introuvable: {PROJECT_PATH}. Montez Drive et vérifiez le chemin.")

# Ajouter au sys.path si nécessaire
if PROJECT_PATH not in sys.path:
    sys.path.insert(0, PROJECT_PATH)
if AI_SUBDIR not in sys.path and os.path.isdir(AI_SUBDIR):
    sys.path.insert(0, AI_SUBDIR)

# Se placer dans le répertoire projet
os.chdir(PROJECT_PATH)

print('Répertoire de travail:', os.getcwd())
print('\nQuelques fichiers à la racine du projet:')
print(sorted(os.listdir(PROJECT_PATH))[:50])
print('\nContenu du dossier ai/:')
print(sorted(os.listdir(AI_SUBDIR))[:100])

# Diagnostic d'import direct pour le module Chess
try:
    import Chess
    print('\n✅ Import direct `Chess` OK (module trouvé via sys.path)')
except Exception as e:
    print('\n❌ Import direct `Chess` a échoué:', e)
    print('Vérifiez que `ai/Chess.py` existe et que le dossier ai/ est dans sys.path')

# Maintenant importer le module d'entraînement (trainer)
try:
    import ai.NN.train_torch as trainer
    import ai.NN.torch_nn_evaluator as torch_eval
    from ai.Chess_v2 import Chess
    print('\n✓ Modules importés avec succès!')
except Exception as e:
    print('\n❌ Erreur d\'import lors de l\'import du trainer:', e)
    raise


Répertoire de travail: /content/drive/MyDrive/smart_chess_drive/smart-chess

Quelques fichiers à la racine du projet:
['.git', '.gitignore', 'README.md', 'ai', 'docs', 'prototypes']

Contenu du dossier ai/:
['AI_reduction', 'Chess.py', 'ChessInteractifv2.py', 'Chess_v2.py', 'NN', 'Null_move_AI', 'Old_AI', 'Player.py', 'Profile', 'Tests.py', '__init__.py', '__pycache__', 'alphabeta.py', 'alphabeta_engine.py', 'alphabeta_engine_v2.py', 'analyze_reduction_overhead.py', 'base_engine.py', 'check_dataset_stats.py', 'check_gpu.py', 'check_performance.py', 'checkpoints', 'chess_model_checkpoint.pt', 'debug_conversion.py', 'engine_match.py', 'evaluator.py', 'example_move_reduction.py', 'fast_evaluator.py', 'journal-experiments.md', 'optimized_chess.py', 'profile_report_1760344602.txt', 'test_depth_6_performance.py', 'test_depth_6_quick.py', 'test_depth_effectiveness.py', 'test_engines_v2.py', 'test_evaluator_performance.py', 'test_generalization.py', 'test_move_reduction.py', 'test_null_move.py

## 7. Configuration de l'entraînement

In [7]:
# Paramètres d'entraînement
CONFIG = {
    # Génération de données
    'num_games': 10000,          # Nombre de parties à générer pour l'entraînement

    # Hyperparamètres
    'batch_size': 256,           # Taille du batch (augmenter si GPU puissant)
    'epochs': 50,                # Nombre d'époques d'entraînement
    'learning_rate': 0.001,      # Taux d'apprentissage

    # Configuration système
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'num_workers': 2,            # Workers pour le DataLoader

    # Sauvegarde
    'checkpoint_path': 'ai/chess_model_checkpoint.pt',
    'save_interval': 5,          # Sauvegarder tous les N époques
}

print("=" * 60)
print("CONFIGURATION DE L'ENTRAÎNEMENT")
print("=" * 60)
for key, value in CONFIG.items():
    print(f"{key:20s}: {value}")
print("=" * 60)

if CONFIG['device'] == 'cpu':
    print("\n⚠️ ATTENTION: Entraînement sur CPU détecté!")
    print("   Réduisez num_games et epochs pour un test rapide.")

CONFIGURATION DE L'ENTRAÎNEMENT
num_games           : 10000
batch_size          : 256
epochs              : 50
learning_rate       : 0.001
device              : cuda
num_workers         : 2
checkpoint_path     : ai/chess_model_checkpoint.pt
save_interval       : 5


## 8. Génération des données d'entraînement

Cette étape génère des parties d'échecs aléatoires et calcule les évaluations de position.
**Attention:** Cela peut prendre 15-30 minutes selon le nombre de parties.

In [8]:
# Localiser le dataset sur Google Drive et préparer le dossier de checkpoints
import os
from glob import glob

# Chemin attendu du dossier contenant le dataset (donné par l'user)
# Updated based on user's feedback that the file is directly in smart_chess_drive
DATASET_DIR = '/content/drive/MyDrive/smart_chess_drive/'

# Chercher un fichier .csv dans DATASET_DIR
DATASET_CSV = None
if os.path.exists(DATASET_DIR):
    csvs = glob(os.path.join(DATASET_DIR, '*.csv'))
    if len(csvs) > 0:
        # Assuming there's only one relevant CSV in that dir, pick the first one
        DATASET_CSV = csvs[0]
        print(f'✅ Dataset CSV trouvé: {DATASET_CSV}')
    else:
        print(f'❌ Aucun fichier .csv trouvé dans {DATASET_DIR}. Placez votre fichier chessData.csv dans ce dossier.')
else:
    print(f'❌ Dossier dataset introuvable: {DATASET_DIR}. Vérifiez le chemin sur votre Drive.')

# Créer un dossier de checkpoints dans le repo sur Drive (persistant)
CKPT_DIR = '/content/drive/MyDrive/smart_chess_drive/smart-chess/ai/checkpoints'
os.makedirs(CKPT_DIR, exist_ok=True)
print('Dossier de checkpoints (créé si manquant):', CKPT_DIR)

# Exposer variables utiles
print('\nVariables exposées:')
print(' DATASET_CSV =', DATASET_CSV)
print(' CKPT_DIR =', CKPT_DIR)

✅ Dataset CSV trouvé: /content/drive/MyDrive/smart_chess_drive/chessData.csv
Dossier de checkpoints (créé si manquant): /content/drive/MyDrive/smart_chess_drive/smart-chess/ai/checkpoints

Variables exposées:
 DATASET_CSV = /content/drive/MyDrive/smart_chess_drive/chessData.csv
 CKPT_DIR = /content/drive/MyDrive/smart_chess_drive/smart-chess/ai/checkpoints


In [9]:
from tqdm import tqdm
import time

print("Chargement du dataset (depuis chessData)...")

# Préférer la variable DATASET_CSV (définie après le montage Drive) sinon utiliser la valeur par défaut du module trainer
dataset_path = globals().get('DATASET_CSV') # Use the DATASET_CSV variable directly

if dataset_path is None:
    raise FileNotFoundError('Aucun chemin de dataset défini. Montez Drive et placez le fichier CSV dans MyDrive/smart_chess_drive/chessData')

start_time = time.time()

# Utiliser la fonction de chargement du script d'entraînement pour assurer le même prétraitement
fens, evaluations = trainer.load_data(dataset_path) # Pass the dataset_path explicitly

# Variables attendues plus bas dans le notebook
X_train = fens
y_train = evaluations

elapsed_time = time.time() - start_time

print("\n" + "=" * 60)
print("DONNÉES CHARGÉES")
print("=" * 60)
print(f"Nombre total de positions: {len(X_train):,}")
print(f"Temps écoulé: {elapsed_time:.1f}s ({elapsed_time/60:.1f} min)")
print("=" * 60)

# Statistiques sur les évaluations
print(f"\nStatistiques sur les évaluations:")
print(f"  Min: {y_train.min():.4f}")
print(f"  Max: {y_train.max():.4f}")
print(f"  Moyenne: {y_train.mean():.4f}")
print(f"  Écart-type: {y_train.std():.4f}")

Chargement du dataset (depuis chessData)...
📂 Chargement du dataset depuis /content/drive/MyDrive/smart_chess_drive/chessData.csv...
🧹 Nettoyage : 190154 lignes corrompues supprimées.
✅ 12,767,881 positions valides chargées.

DONNÉES CHARGÉES
Nombre total de positions: 12,767,881
Temps écoulé: 20.9s (0.3 min)

Statistiques sur les évaluations:
  Min: -15.3120
  Max: 15.3190
  Moyenne: 0.0455
  Écart-type: 0.8139


In [10]:
import inspect
import ai.NN.train_torch as trainer

try:
    # Get the source code of the load_data function
    source_code = inspect.getsource(trainer.load_data)
    print("Source code of trainer.load_data:")
    print("=" * 60)
    print(source_code)
    print("=" * 60)
except TypeError:
    print("Could not get source code for trainer.load_data. It might not be a function defined in the file.")
except FileNotFoundError:
    print("Could not find the train_torch.py file.")
except Exception as e:
    print(f"An error occurred while trying to get source code: {e}")

Source code of trainer.load_data:
def load_data(filepath: str):
    """Charge le dataset FEN,Evaluation et le nettoie."""
    print(f"📂 Chargement du dataset depuis {filepath}...")
    
    df = pd.read_csv(
        filepath, 
        names=['FEN', 'Evaluation'], 
        skiprows=1,
        comment='#'
    )
    
    initial_count = len(df)
    df.dropna(inplace=True)
    cleaned_count = len(df)
    
    if initial_count > cleaned_count:
        print(f"🧹 Nettoyage : {initial_count - cleaned_count} lignes corrompues supprimées.")
    
    fens = df['FEN'].values
    EVAL_SCALE_FACTOR = 1000.0
    evaluations = (df['Evaluation'].astype(int).values) / EVAL_SCALE_FACTOR
    
    print(f"✅ {len(fens):,} positions valides chargées.")
    return fens, evaluations



In [11]:
import os

file_path = os.path.join(PROJECT_PATH, 'ai/NN/train_torch.py')

# Read the content of the file
with open(file_path, 'r') as f:
    content = f.read()

# Assuming the load_data function signature is currently load_data():
# We need to find the function definition and modify it to accept dataset_path
# This is a simple string replacement and might need adjustment based on the actual code
old_def = 'def load_data():'
new_def = 'def load_data(dataset_path):'
old_data_loading_line = "df = pd.read_csv('C:\\\\Users\\\\gauti\\\\OneDrive\\\\Documents\\\\UE commande\\\\chessData.csv')" # This is a guess, may need adjustment
new_data_loading_line = "df = pd.read_csv(dataset_path)"


if old_def in content and old_data_loading_line in content:
    content = content.replace(old_def, new_def)
    content = content.replace(old_data_loading_line, new_data_loading_line)
    # Write the modified content back to the file
    with open(file_path, 'w') as f:
        f.write(content)
    print(f"Successfully modified {file_path} to accept and use dataset_path in load_data function.")
elif old_def in content:
     print(f"Found function definition '{old_def}', but could not find the specific data loading line '{old_data_loading_line}' to replace.")
     print("Please inspect the `load_data` function in `ai/NN/train_torch.py` and manually update the file path to use the `dataset_path` argument.")
else:
    print(f"Could not find the function definition '{old_def}' in {file_path}. Please inspect the file manually.")

Could not find the function definition 'def load_data():' in /content/drive/MyDrive/smart_chess_drive/smart-chess/ai/NN/train_torch.py. Please inspect the file manually.


## 9. Création du dataset et du dataloader

In [12]:
from torch.utils.data import DataLoader
from ai.NN.train_torch import ChessDataset # Import ChessDataset

# Créer le dataset
dataset = ChessDataset(X_train, y_train)

# Créer le dataloader
train_loader = DataLoader(
    dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=True,
    num_workers=CONFIG['num_workers'],
    pin_memory=True if CONFIG['device'] == 'cuda' else False
)

print("=" * 60)
print("DATALOADER CONFIGURÉ")
print("=" * 60)
print(f"Taille du dataset: {len(dataset):,} échantillons")
print(f"Nombre de batches: {len(train_loader):,}")
print(f"Taille du batch: {CONFIG['batch_size']}")
print(f"Dernière batch: {len(dataset) % CONFIG['batch_size']} échantillons")
print("=" * 60)

DATALOADER CONFIGURÉ
Taille du dataset: 12,767,881 échantillons
Nombre de batches: 49,875
Taille du batch: 256
Dernière batch: 137 échantillons


## 10. Création du modèle

In [13]:
# Créer le modèle et le déplacer sur le device approprié
from ai.NN.torch_nn_evaluator import TorchNNEvaluator # Import TorchNNEvaluator from torch_nn_evaluator

model = TorchNNEvaluator().to(CONFIG['device'])

# Afficher l'architecture
print("=" * 60)
print("ARCHITECTURE DU MODÈLE")
print("=" * 60)
print(model)
print("=" * 60)

# Compter les paramètres
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\nNombre total de paramètres: {total_params:,}")
print(f"Paramètres entraînables: {trainable_params:,}")
print(f"Device: {CONFIG['device']}")

# Estimer la taille mémoire du modèle
param_size_mb = total_params * 4 / (1024 ** 2)  # 4 bytes par float32
print(f"Taille estimée du modèle: {param_size_mb:.2f} MB")

ARCHITECTURE DU MODÈLE
TorchNNEvaluator(
  (l1): Linear(in_features=768, out_features=256, bias=True)
  (l2): Linear(in_features=256, out_features=256, bias=True)
  (l3): Linear(in_features=256, out_features=1, bias=True)
  (dropout1): Dropout(p=0.3, inplace=False)
  (dropout2): Dropout(p=0.3, inplace=False)
  (leaky_relu): LeakyReLU(negative_slope=0.01)
)

Nombre total de paramètres: 262,913
Paramètres entraînables: 262,913
Device: cuda
Taille estimée du modèle: 1.00 MB


## 11. Entraînement du modèle

Cette étape lance l'entraînement complet. Les checkpoints sont sauvegardés automatiquement sur votre Drive.

In [18]:
# @title
# Configurer et lancer le script d'entraînement `ai.NN.train_torch` en adaptant les chemins pour Colab/Drive
import os
import importlib

if DATASET_CSV is None:
    raise FileNotFoundError(f"Dataset non trouvé dans: {DATASET_DIR}")

# Importer le module d'entraînement
import ai.NN.train_torch as trainer

# Reload the module to pick up recent changes
importlib.reload(trainer)


# Rediriger les chemins dataset et checkpoints vers Drive
trainer.DATASET_PATH = DATASET_CSV
trainer.CHECKPOINT_FILE = os.path.join(CKPT_DIR, os.path.basename(trainer.CHECKPOINT_FILE))
trainer.WEIGHTS_FILE = os.path.join(CKPT_DIR, os.path.basename(trainer.WEIGHTS_FILE))

# Harmonisation des tailles de batch (Option B):
# On force le module trainer à utiliser la valeur définie dans CONFIG['batch_size']
try:
    trainer.BATCH_SIZE = CONFIG['batch_size']
    print(f'✅ Harmonisation: trainer.BATCH_SIZE = {trainer.BATCH_SIZE}')
except Exception as e:
    print('⚠️ Impossible de définir trainer.BATCH_SIZE:', e)

# --- Apply user-requested global hyperparameter changes ---
# Increase hidden layer size to 512 and set per-epoch samples to 200k
try:
    trainer.HIDDEN_SIZE = 512
    trainer.MAX_SAMPLES = 200_000
    print(f"✅ Applied global changes: trainer.HIDDEN_SIZE={trainer.HIDDEN_SIZE}, trainer.MAX_SAMPLES={trainer.MAX_SAMPLES}")
except Exception as e:
    print('⚠️ Impossible de définir trainer.HIDDEN_SIZE / trainer.MAX_SAMPLES:', e)

# Optionally set other CONFIG parameters from the notebook if needed
# trainer.EPOCHS = CONFIG['epochs']
# trainer.LEARNING_RATE = CONFIG['learning_rate']
# trainer.DEVICE = CONFIG['device']

# Optionnel: réduire pour test rapide (décommentez si besoin)
# trainer.EPOCHS = 2
# trainer.MAX_SAMPLES = 5000

print('Configuration trainer:')
print(' DATASET_PATH=', trainer.DATASET_PATH)
print(' CHECKPOINT_FILE=', trainer.CHECKPOINT_FILE)
print(' WEIGHTS_FILE=', trainer.WEIGHTS_FILE)
print(' EPOCHS=', trainer.EPOCHS)
print(' MAX_SAMPLES=', trainer.MAX_SAMPLES)


# Lancer l'entraînement
trainer.main()


🖥️  Device: cuda
🚀 GPU: NVIDIA L4
💾 GPU Memory: 23.80 GB
✅ Harmonisation: trainer.BATCH_SIZE = 256
✅ Applied global changes: trainer.HIDDEN_SIZE=512, trainer.MAX_SAMPLES=200000
Configuration trainer:
 DATASET_PATH= /content/drive/MyDrive/smart_chess_drive/chessData.csv
 CHECKPOINT_FILE= /content/drive/MyDrive/smart_chess_drive/smart-chess/ai/checkpoints/chess_model_checkpoint.pt
 WEIGHTS_FILE= /content/drive/MyDrive/smart_chess_drive/smart-chess/ai/checkpoints/chess_nn_weights.npz
 EPOCHS= 20
 MAX_SAMPLES= 200000
📂 Chargement du dataset depuis /content/drive/MyDrive/smart_chess_drive/chessData.csv...
🧹 Nettoyage : 190154 lignes corrompues supprimées.
✅ 12,767,881 positions valides chargées.

📊 Dataset complet: 12,767,881 positions
📥 Chargement du checkpoint PyTorch: /content/drive/MyDrive/smart_chess_drive/smart-chess/ai/checkpoints/chess_model_checkpoint.pt
✅ Checkpoint chargé (step 20), best_rmse=0.5130490660667419

Configuration:
  Dataset complet: 12,767,881 positions
  Échantillon

Epoch 1/20:   1%|          | 7/782 [00:00<00:12, 61.88it/s, loss=0.5850]


[DEBUG batch 0] targets mean=0.0136 std=0.5158; preds mean=0.0371 std=0.3577; RMSE=0.4076; corr=0.6191


Epoch 1/20: 100%|██████████| 782/782 [00:11<00:00, 69.55it/s, loss=0.5848]



🔍 Évaluation epoch 1...

EPOCH 1/20 - Évaluation sur 5,000 positions
  RMSE:        0.5679  (baseline: 0.7983)
  MAE:         0.2374
  Amélioration: +28.9% vs baseline
  Corrélation: 0.7151
  Std preds:   0.4657  (cible: 0.7983)
  Mean preds:  0.0371  (cible: 0.0384)
  →  Apprentissage en cours


[Epoch 2] 🎲 Échantillonnage: 200,000 positions sur 12,767,881
➡️ Learning rate courant: 0.000002


...


🔍 Évaluation epoch 18...

EPOCH 18/20 - Évaluation sur 5,000 positions
  RMSE:        0.5879  (baseline: 0.8571)
  MAE:         0.2410
  Amélioration: +31.4% vs baseline
  Corrélation: 0.7396
  Std preds:   0.5212  (cible: 0.8571)
  Mean preds:  0.0458  (cible: 0.0576)
  ✓  Bon apprentissage!


[Epoch 19] 🎲 Échantillonnage: 200,000 positions sur 12,767,881
➡️ Learning rate courant: 0.000000


Epoch 19/20: 100%|██████████| 782/782 [00:11<00:00, 69.58it/s, loss=0.5880]



🔍 Évaluation epoch 19...

EPOCH 19/20 - Évaluation sur 5,000 positions
  RMSE:        0.5327  (baseline: 0.7637)
  MAE:         0.2295
  Amélioration: +30.3% vs baseline
  Corrélation: 0.7182
  Std preds:   0.5124  (cible: 0.7637)
  Mean preds:  0.0453  (cible: 0.0502)
  ✓  Bon apprentissage!


[Epoch 20] 🎲 Échantillonnage: 200,000 positions sur 12,767,881
➡️ Learning rate courant: 0.000000


Epoch 20/20: 100%|██████████| 782/782 [00:11<00:00, 68.14it/s, loss=0.5937]



🔍 Évaluation epoch 20...

EPOCH 20/20 - Évaluation sur 5,000 positions
  RMSE:        0.5439  (baseline: 0.8142)
  MAE:         0.2378
  Amélioration: +33.2% vs baseline
  Corrélation: 0.7549
  Std preds:   0.5112  (cible: 0.8142)
  Mean preds:  0.0359  (cible: 0.0353)
  ✓  Bon apprentissage!


🎉 Entraînement terminé!
📊 Meilleur RMSE: 0.5130

💾 Sauvegarde finale...
Checkpoint PyTorch sauvegardé dans /content/drive/MyDrive/smart_chess_drive/smart-chess/ai/checkpoints/chess_model_checkpoint.pt
Poids et moments Adam sauvegardés (npz) dans /content/drive/MyDrive/smart_chess_drive/smart-chess/ai/checkpoints/chess_nn_weights.npz
✅ Modèle sauvegardé dans /content/drive/MyDrive/smart_chess_drive/smart-chess/ai/checkpoints/chess_model_checkpoint.pt et /content/drive/MyDrive/smart_chess_drive/smart-chess/ai/checkpoints/chess_nn_weights.npz


## 12. Visualisation des résultats

In [ ]:
# @title
import matplotlib.pyplot as plt

# Configurer le style des graphiques
plt.style.use('seaborn-v0_8-darkgrid')
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Graphique 1: Loss
axes[0].plot(history['loss'], linewidth=2, color='#2E86AB', label='Training Loss')
axes[0].set_xlabel('Époque', fontsize=12)
axes[0].set_ylabel('Loss (MSE)', fontsize=12)
axes[0].set_title('Évolution de la perte pendant l\'entraînement', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Afficher les valeurs min/max
min_loss = min(history['loss'])
max_loss = max(history['loss'])
axes[0].axhline(y=min_loss, color='green', linestyle='--', alpha=0.5, label=f'Min: {min_loss:.6f}')
axes[0].legend(fontsize=10)

# Graphique 2: MAE (si disponible)
if 'mae' in history:
    axes[1].plot(history['mae'], linewidth=2, color='#F77F00', label='MAE')
    axes[1].set_xlabel('Époque', fontsize=12)
    axes[1].set_ylabel('MAE', fontsize=12)
    axes[1].set_title('Erreur absolue moyenne', fontsize=14, fontweight='bold')
    axes[1].legend(fontsize=10)
    axes[1].grid(True, alpha=0.3)

    min_mae = min(history['mae'])
    axes[1].axhline(y=min_mae, color='green', linestyle='--', alpha=0.5, label=f'Min: {min_mae:.6f}')
    axes[1].legend(fontsize=10)
else:
    axes[1].text(0.5, 0.5, 'MAE non disponible',
                ha='center', va='center', fontsize=14, transform=axes[1].transAxes)
    axes[1].set_xticks([])
    axes[1].set_yticks([])

plt.tight_layout()
plt.savefig('training_history.png', dpi=150, bbox_inches='tight')
plt.show()

# Afficher les statistiques finales
print("\n" + "=" * 60)
print("STATISTIQUES FINALES")
print("=" * 60)
print(f"Perte finale: {history['loss'][-1]:.6f}")
print(f"Perte minimale: {min_loss:.6f} (époque {history['loss'].index(min_loss) + 1})")
if 'mae' in history:
    print(f"MAE final: {history['mae'][-1]:.6f}")
    print(f"MAE minimal: {min_mae:.6f} (époque {history['mae'].index(min_mae) + 1})")
print("=" * 60)

## 13. Sauvegarde du modèle final

In [ ]:
# @title
import datetime

# Timestamp pour identifier cette sauvegarde
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# Sauvegarder le modèle complet avec l'historique
final_model_path = f'ai/chess_model_final_{timestamp}.pt'
torch.save({
    'epoch': CONFIG['epochs'],
    'model_state_dict': model.state_dict(),
    'config': CONFIG,
    'history': history,
    'timestamp': timestamp,
}, final_model_path)

print("=" * 60)
print("SAUVEGARDE DES MODÈLES")
print("=" * 60)
print(f"✓ Modèle final: {final_model_path}")

# Sauvegarder aussi au format .npz pour compatibilité avec l'ancien code
weights_path = 'ai/NN/chess_nn_weights.npz'
weights = {name: param.cpu().detach().numpy() for name, param in model.named_parameters()}
np.savez(weights_path, **weights)
print(f"✓ Poids .npz: {weights_path}")

# Copier aussi le checkpoint dans NN/
import shutil
checkpoint_backup = f'ai/NN/chess_model_checkpoint_{timestamp}.pt'
if os.path.exists(CONFIG['checkpoint_path']):
    shutil.copy(CONFIG['checkpoint_path'], checkpoint_backup)
    print(f"✓ Checkpoint backup: {checkpoint_backup}")

print("=" * 60)
print("\n✅ Tous les fichiers sont sauvegardés sur votre Google Drive!")
print(f"   Chemin: {PROJECT_PATH}")

## 14. Test du modèle sur des positions aléatoires

In [ ]:
# @title
# Passer le modèle en mode évaluation
model.eval()

# Tester sur quelques positions aléatoires
num_tests = 10
test_indices = np.random.choice(len(X_train), num_tests, replace=False)

print("=" * 60)
print(f"TEST SUR {num_tests} POSITIONS ALÉATOIRES")
print("=" * 60)

errors = []

with torch.no_grad():
    for i, idx in enumerate(test_indices, 1):
        x = torch.FloatTensor(X_train[idx:idx+1]).to(CONFIG['device'])
        y_true = y_train[idx]
        y_pred = model(x).cpu().numpy()[0, 0]
        error = abs(y_true - y_pred)
        errors.append(error)

        print(f"\nPosition {i}:")
        print(f"  Évaluation réelle:  {y_true:+8.4f}")
        print(f"  Prédiction modèle:  {y_pred:+8.4f}")
        print(f"  Erreur absolue:     {error:8.4f}")

        # Indicateur visuel de la qualité
        if error < 0.1:
            print(f"  Qualité: ✅ Excellente")
        elif error < 0.3:
            print(f"  Qualité: ✓ Bonne")
        elif error < 0.5:
            print(f"  Qualité: ⚠ Moyenne")
        else:
            print(f"  Qualité: ❌ Faible")

print("\n" + "=" * 60)
print("STATISTIQUES DES TESTS")
print("=" * 60)
print(f"Erreur moyenne: {np.mean(errors):.4f}")
print(f"Erreur médiane: {np.median(errors):.4f}")
print(f"Erreur min:     {np.min(errors):.4f}")
print(f"Erreur max:     {np.max(errors):.4f}")
print(f"Écart-type:     {np.std(errors):.4f}")
print("=" * 60)

## 15. Résumé et fichiers générés

In [ ]:
# @title
print("\n" + "="*60)
print("📊 RÉSUMÉ DE L'ENTRAÎNEMENT")
print("="*60)
print(f"\n📍 Projet: {PROJECT_PATH}")
print(f"\n⚙️ Configuration:")
print(f"   • Parties générées: {CONFIG['num_games']:,}")
print(f"   • Positions d'entraînement: {len(X_train):,}")
print(f"   • Époques: {CONFIG['epochs']}")
print(f"   • Batch size: {CONFIG['batch_size']}")
print(f"   • Learning rate: {CONFIG['learning_rate']}")
print(f"   • Device: {CONFIG['device']}")

print(f"\n📈 Résultats:")
print(f"   • Perte finale: {history['loss'][-1]:.6f}")
print(f"   • Perte minimale: {min(history['loss']):.6f}")
if 'mae' in history:
    print(f"   • MAE final: {history['mae'][-1]:.6f}")

print(f"\n💾 Fichiers sauvegardés sur Drive:")
files_to_check = [
    final_model_path,
    CONFIG['checkpoint_path'],
    weights_path,
    'training_history.png'
]

for filepath in files_to_check:
    if os.path.exists(filepath):
        size = os.path.getsize(filepath) / (1024 * 1024)  # Convertir en MB
        print(f"   ✓ {filepath} ({size:.2f} MB)")
    else:
        print(f"   ✗ {filepath} (non trouvé)")

print("\n" + "="*60)
print("✅ ENTRAÎNEMENT TERMINÉ AVEC SUCCÈS!")
print("="*60)
print("\nTous les fichiers sont automatiquement synchronisés avec votre Google Drive.")
print("Vous pouvez fermer ce notebook en toute sécurité.\n")

In [ ]:
# @title
import os

checkpoint_file = '/content/drive/MyDrive/smart_chess_drive/smart-chess/ai/checkpoints/chess_model_checkpoint.pt'

if os.path.exists(checkpoint_file):
    print(f"Removing existing checkpoint file: {checkpoint_file}")
    os.remove(checkpoint_file)
    print("Checkpoint removed.")
else:
    print(f"No checkpoint file found at {checkpoint_file}. No action needed.")

In [ ]:
# @title
# Smoke tests automatisés — 3 runs courts pour comparer configurations
# - Crée une validation fixe, lance 3 expériences courtes (EPOCHS=3, MAX_SAMPLES=100k)
# - Sauvegarde checkpoints séparés et évalue chaque modèle sur la validation fixe

import time
import importlib
import os
import numpy as np
from torch.utils.data import DataLoader

print('Lancement des smoke tests (rapides).\n')

# Vérifier dataset et données en mémoire
if 'X_train' not in globals() or 'y_train' not in globals():
    print('X_train/y_train non trouvés en mémoire — chargement léger depuis trainer.DATASET_PATH (peut prendre du temps)...')
    fens, evaluations = trainer.load_data(trainer.DATASET_PATH)
    X_train = fens
    y_train = evaluations

# Créer validation fixe (seed deterministe)
val_size = min(5000, len(X_train))
rs = np.random.RandomState(42)
val_idx = rs.choice(len(X_train), size=val_size, replace=False)
val_fens = X_train[val_idx]
val_targets = y_train[val_idx]
print(f'Validation fixe : {val_size} positions (seed=42)')

# Sauvegarder originaux pour restauration
orig_keys = ['HIDDEN_SIZE','DROPOUT','LEARNING_RATE','WEIGHT_DECAY','BATCH_SIZE','EPOCHS','MAX_SAMPLES','CHECKPOINT_FILE','WEIGHTS_FILE','EVAL_MAX_SAMPLES']
orig = {k: getattr(trainer, k) for k in orig_keys if hasattr(trainer, k)}

# Expériences à tester (changes applied on top of orig)
experiments = [
    {'name': 'baseline', 'HIDDEN_SIZE': orig.get('HIDDEN_SIZE', 256), 'DROPOUT': orig.get('DROPOUT', 0.3), 'LEARNING_RATE': orig.get('LEARNING_RATE', 0.001)},
    {'name': 'bigger', 'HIDDEN_SIZE': 512, 'DROPOUT': 0.2, 'LEARNING_RATE': 5e-4},
    {'name': 'smaller_lr', 'HIDDEN_SIZE': 512, 'DROPOUT': 0.2, 'LEARNING_RATE': 1e-4},
]

results = []

for exp in experiments:
    print('\n' + '='*80)
    print(f"Exp: {exp['name']}")
    print('='*80)

    # Set quick test params
    trainer.EPOCHS = 3
    trainer.MAX_SAMPLES = 100_000
    trainer.EVAL_MAX_SAMPLES = 2000

    # Apply experiment overrides
    trainer.HIDDEN_SIZE = exp['HIDDEN_SIZE']
    trainer.DROPOUT = exp['DROPOUT']
    trainer.LEARNING_RATE = exp['LEARNING_RATE']

    # Use separate checkpoint/weights files to avoid overwriting
    ckpt_path = os.path.join(CKPT_DIR, f"smoke_{exp['name']}.pt")
    weights_path = os.path.join(CKPT_DIR, f"smoke_{exp['name']}.npz")
    trainer.CHECKPOINT_FILE = ckpt_path
    trainer.WEIGHTS_FILE = weights_path

    print('Parameters:')
    print(f" HIDDEN_SIZE={trainer.HIDDEN_SIZE}, DROPOUT={trainer.DROPOUT}, LR={trainer.LEARNING_RATE}")
    print(f" EPOCHS={trainer.EPOCHS}, MAX_SAMPLES={trainer.MAX_SAMPLES}")
    print(f" CHECKPOINT -> {trainer.CHECKPOINT_FILE}")

    # Run training (blocking)
    t0 = time.time()
    try:
        trainer.main()
    except Exception as e:
        print('Erreur pendant trainer.main():', e)
        # continue to evaluation attempt (if checkpoint exists)
    t1 = time.time()
    print(f"Run time: {t1-t0:.1f}s")

    # Load model from checkpoint
    try:
        model = trainer.TorchNNEvaluator(hidden_size=trainer.HIDDEN_SIZE, dropout=trainer.DROPOUT, leaky_alpha=trainer.LEAKY_ALPHA)
        optimizer = trainer.optim.AdamW(model.parameters(), lr=trainer.LEARNING_RATE, weight_decay=trainer.WEIGHT_DECAY)
        model, optim_state, step = trainer.torch_load_checkpoint(trainer.CHECKPOINT_FILE, model, optimizer, device=trainer.DEVICE)

        # Evaluation on fixed val set
        eval_dataset = trainer.ChessDataset(val_fens, val_targets)
        eval_loader = DataLoader(eval_dataset, batch_size=max(1, trainer.BATCH_SIZE//2), shuffle=False)
        rmse, mae, corr, preds, targets = trainer.evaluate_model(model, eval_loader, trainer.DEVICE)
        print(f"Eval results — RMSE: {rmse:.4f}, MAE: {mae:.4f}, Corr: {corr:.4f}")
        results.append({'exp': exp['name'], 'rmse': rmse, 'mae': mae, 'corr': corr})
    except FileNotFoundError:
        print('Checkpoint not found, skipping evaluation for this experiment.')
        results.append({'exp': exp['name'], 'rmse': None, 'mae': None, 'corr': None})
    except Exception as e:
        print('Erreur lors de l\'évaluation:', e)
        results.append({'exp': exp['name'], 'rmse': None, 'mae': None, 'corr': None})

# Restore original trainer settings
for k, v in orig.items():
    setattr(trainer, k, v)

print('\n' + '='*80)
print('Résumé des smoke tests:')
for r in results:
    print(r)
print('='*80)

# End of smoke tests


In [ ]:
# @title
# Smoke-extended paramétrable — runs répétés et mode rapide
# Usage:
# - régler FAST_MODE=True pour itérations ultra-rapides (EPOCHS=1, MAX_SAMPLES=20k)
# - régler REPS pour répéter chaque expérience sur plusieurs seeds

import time
import os
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader

print('\n=== Smoke-extended démarré ===')

# Paramètres utilisateur
FAST_MODE = False        # True -> very fast (useful for quick checks)
EPOCHS_EXT = 5
MAX_SAMPLES_EXT = 200_000
REPS = 1                # nombre de répétitions par configuration
BASE_SEED = 42

if FAST_MODE:
    EPOCHS_EXT = 1
    MAX_SAMPLES_EXT = 20_000

print(f"FAST_MODE={FAST_MODE}, EPOCHS={EPOCHS_EXT}, MAX_SAMPLES={MAX_SAMPLES_EXT}, REPS={REPS}")

# Vérifier que trainer est chargé
if 'trainer' not in globals():
    raise RuntimeError('Le module trainer (ai.NN.train_torch) doit être importé avant d\'exécuter cette cellule.')

# Charger données si nécessaire
if 'X_train' not in globals() or 'y_train' not in globals():
    print('X_train/y_train non présents en mémoire — chargement via trainer.load_data(...) (peut être long)')
    fens, evaluations = trainer.load_data(trainer.DATASET_PATH)
    X_train = fens
    y_train = evaluations

# Création d'une validation fixe
val_size = min(5000, len(X_train))
rs = np.random.RandomState(42)
val_idx = rs.choice(len(X_train), size=val_size, replace=False)
val_fens = X_train[val_idx]
val_targets = y_train[val_idx]
print(f'Validation fixe : {val_size} positions (seed=42)')

# Conserver paramètres originaux
orig_keys = ['HIDDEN_SIZE','DROPOUT','LEARNING_RATE','WEIGHT_DECAY','BATCH_SIZE','EPOCHS','MAX_SAMPLES','CHECKPOINT_FILE','WEIGHTS_FILE','EVAL_MAX_SAMPLES']
orig = {k: getattr(trainer, k) for k in orig_keys if hasattr(trainer, k)}

# Expériences à tester
experiments = [
    {'name': 'baseline', 'HIDDEN_SIZE': orig.get('HIDDEN_SIZE', 256), 'DROPOUT': orig.get('DROPOUT', 0.3), 'LEARNING_RATE': orig.get('LEARNING_RATE', 0.001)},
    {'name': 'bigger', 'HIDDEN_SIZE': 512, 'DROPOUT': 0.2, 'LEARNING_RATE': 5e-4},
    {'name': 'smaller_lr', 'HIDDEN_SIZE': 512, 'DROPOUT': 0.2, 'LEARNING_RATE': 1e-4},
]

results = []

for exp in experiments:
    for rep in range(REPS):
        seed = BASE_SEED + rep
        run_name = f"{exp['name']}_r{rep+1}_s{seed}"
        print('\n' + '='*80)
        print(f"Run: {run_name}")
        print('='*80)

        # appliquer paramètres rapides
        trainer.EPOCHS = EPOCHS_EXT
        trainer.MAX_SAMPLES = MAX_SAMPLES_EXT
        trainer.EVAL_MAX_SAMPLES = min(2000, MAX_SAMPLES_EXT//50)

        # appliquer overrides de l'expérience
        trainer.HIDDEN_SIZE = exp['HIDDEN_SIZE']
        trainer.DROPOUT = exp['DROPOUT']
        trainer.LEARNING_RATE = exp['LEARNING_RATE']

        # checkpoints séparés
        ckpt_path = os.path.join(CKPT_DIR, f"smoke_ext_{run_name}.pt")
        weights_path = os.path.join(CKPT_DIR, f"smoke_ext_{run_name}.npz")
        trainer.CHECKPOINT_FILE = ckpt_path
        trainer.WEIGHTS_FILE = weights_path

        print('Params:', f"H={trainer.HIDDEN_SIZE}, dropout={trainer.DROPOUT}, lr={trainer.LEARNING_RATE}")
        print('Run params:', f"EPOCHS={trainer.EPOCHS}, MAX_SAMPLES={trainer.MAX_SAMPLES}")
        print('Checkpoint:', trainer.CHECKPOINT_FILE)

        # Fix seeds where possible
        try:
            import torch
            np.random.seed(seed)
            torch.manual_seed(seed)
            if trainer.DEVICE and 'cuda' in str(trainer.DEVICE):
                torch.cuda.manual_seed_all(seed)
        except Exception:
            pass

        t0 = time.time()
        try:
            trainer.main()
        except Exception as e:
            print('Erreur pendant trainer.main():', e)
        t1 = time.time()
        print(f'Run time: {t1-t0:.1f}s')

        # Évaluation sur la validation fixe
        try:
            model = trainer.TorchNNEvaluator(hidden_size=trainer.HIDDEN_SIZE, dropout=trainer.DROPOUT, leaky_alpha=getattr(trainer,'LEAKY_ALPHA',0.01))
            optimizer = trainer.optim.AdamW(model.parameters(), lr=trainer.LEARNING_RATE, weight_decay=getattr(trainer,'WEIGHT_DECAY',1e-4))
            model, opt_state, step = trainer.torch_load_checkpoint(trainer.CHECKPOINT_FILE, model, optimizer, device=trainer.DEVICE)

            eval_dataset = trainer.ChessDataset(val_fens, val_targets)
            eval_loader = DataLoader(eval_dataset, batch_size=max(1, trainer.BATCH_SIZE//2), shuffle=False)
            rmse, mae, corr, preds, targets = trainer.evaluate_model(model, eval_loader, trainer.DEVICE)
            print(f"Eval — RMSE: {rmse:.4f}, MAE: {mae:.4f}, Corr: {corr:.4f}")
            results.append({'run': run_name, 'exp': exp['name'], 'seed': seed, 'rmse': rmse, 'mae': mae, 'corr': corr, 'ckpt': trainer.CHECKPOINT_FILE})
        except FileNotFoundError:
            print('Checkpoint introuvable, évaluation sautée.')
            results.append({'run': run_name, 'exp': exp['name'], 'seed': seed, 'rmse': None, 'mae': None, 'corr': None, 'ckpt': trainer.CHECKPOINT_FILE})
        except Exception as e:
            print('Erreur pendant l\'évaluation:', e)
            results.append({'run': run_name, 'exp': exp['name'], 'seed': seed, 'rmse': None, 'mae': None, 'corr': None, 'ckpt': trainer.CHECKPOINT_FILE})

# Restaurer paramètres d'origine
for k, v in orig.items():
    setattr(trainer, k, v)

print('\n=== Résumé smoke-extended ===')
df = pd.DataFrame(results)
print(df)

# Sauvegarder résumé csv
summary_path = os.path.join(CKPT_DIR, 'smoke_extended_summary.csv')
df.to_csv(summary_path, index=False)
print('Résumé sauvegardé dans', summary_path)
